In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [78]:
# Residual for single point
def r(A, c, zi, wi):
    v = zi - c
    x = v.T @ A @ v
    return (wi == 1) * np.maximum(x - 1, 0) + (wi == -1) * np.maximum(1 - x, 0)


# Residual vector
def R(A, c, z, w):
    R = np.zeros_like(w)
    for i in range(R.size):
        R[i] = r(A, c, z[i], w[i])
    return R


# Objective function
def f(R):
    return np.sum(R**2)


# Gradient of objective function. Returns (n+k) vector
def grad_f(R):
    J = jacobian_R(A, c, z)
    return J.T @ R

In [79]:
# Insert coefficients in (nxn) matrix A, given (m+k) vector x
def update_A(A, x):
    n = A.shape[0]
    a = x[:n*(n+1)//2]
    for j in range(n):
        start = j*n - j*(j+1)//n
        end = start + n-j
        A[j, j:] = x[start:end]
    return A

# Insert coefficients in (n) centering vector given (n+k) vector x
def update_c(c, x):
    c = x[-n:]
    return c

# Get the first (k) coefficients of (n+k) vector x, given an (nxn) matrix A
def extract_a(A):
    n = A.shape[0]
    k = n*(n+1)//2
    a = np.zeros(k)
    end = 0
    
    for j in range(n):
        start = end
        end = start + n-j
        a[start:end] = A[j, j:]
        
    return a

In [80]:
# return partial derivatives of residual i, wrt. the matrix coefficients
def r_delA(A, c, zi):
    v = zi - c
    rdA = 2 * (v * (v * np.ones_like(A)).T)
    rdA = rdA - 1/2 * np.identity(c.size) * rdA
    return extract_a(rdA)

# return partial derivatives of residual i, wrt. centering vector coefficients
def r_delc(A, c, zi):
    v = zi - c
    n = c.size
    return - 2 * (v.T @ A)

# Return vector of all partial derivatives of residual i
def del_r(A, c, zi):
    v = zi - c
    n = np.size(c)
    k = n*(n+1)//2
    dr = np.zeros(k+n)
    dr[:k] = r_delA(A, c, zi)
    dr[k:] = r_delc(A, c, zi)
    return dr

#Jacobian of a all residual points. 
def jacobian_R(A, c, z):
    m = z.shape[0]
    n = c.size
    k = n*(n+1)//2
    R = np.zeros((m, n+k))
    for i in range(m):
        R[i] = del_r(A, c, z[i])
    return R

In [81]:
A = np.array((1, 2, 3, 2, 4, 5, 3, 5, 6)).reshape(3, 3)
z = np.array((2, 3, 2, 4, 4, 5)).reshape(2,3)
c = np.array((1, 4, 5))
w = np.array((1, 1))

In [82]:
v = z - c
v

array([[ 1, -1, -3],
       [ 3,  0,  0]])

In [83]:
R = R(A, c, z, w)
J_R = jacobian_R(A, c, z)

In [84]:
R

array([66,  8])

In [85]:
J_R

array([[  1.,  -2.,  -6.,   1.,   6.,   9.,  20.,  34.,  40.],
       [  9.,   0.,   0.,   0.,   0.,   0.,  -6., -12., -18.]])

In [86]:
grad_f(R)

array([  138.,  -132.,  -396.,    66.,   396.,   594.,  1272.,  2148.,
        2496.])